In [1]:
### UTILS
def get_input(dayNum):
    import os
    data_file = f"./data/day-{dayNum}.input"
    if not os.path.exists(data_file):
        download_input(dayNum) 
    return [line.strip() for line in open(data_file).readlines()]

def download_input(dayNum):
    import subprocess
    SESSION_COOKIE = open('./session_cookie.secret').readlines()[0].strip()
    url = f"https://adventofcode.com/2020/day/{dayNum}/input"
    cmd = f"curl -H 'Cookie: session={SESSION_COOKIE}' {url} > ./data/day-{dayNum}.input"
    subprocess.run(cmd, capture_output=True, shell=True)

def manhattan_distance(pointA,pointB):
    return abs(pointA[0] - pointB[0]) + abs(pointA[1] - pointB[1])
    
# input is e.g. [0,1,0,0]
# output is e.g 0b0100 -> 4
def bits_to_int(bits):
    return int("0b" + "".join([str(b) for b in bits]), base=2)

assert(bits_to_int([0,1,0,0]) == 4)

# https://en.wikipedia.org/wiki/Extended_Euclidean_algorithm#Example
def extended_gcd(a, b):
    (old_r, r) = (a,b)
    (old_s, s) = (1,0)
    (old_t,t)  = (0,1)

    i = -1
    while r != 0:
        i += 1
        quotient = old_r // r
        (old_r, r) = (r, old_r - quotient * r)
        (old_s, s) = (s, old_s - quotient * s)
        (old_t, t) = (t, old_t - quotient * t)
    
    return {'bezout': (old_s, old_t), 'gcd': old_r, 'quotients': (t, s)}

def bezout_coeff(a,b):
    return extended_gcd(a,b)['bezout']

bezout_coeff(3,4)

(-1, 1)

In [2]:
# https://adventofcode.com/2020/day/4
def Day4():
    def get_passports():
        passports = []
        lines = get_input(4)
        cur_passport = {}
        for line in lines:
            if line == "":
                if cur_passport:
                    passports.append(cur_passport)
                cur_passport = {}
            field_pairs = line.split()
            for pair in field_pairs:
                [key,val] = pair.split(':')
                cur_passport[key] = val
        passports.append(cur_passport)
        return passports
        
    passports = get_passports()
    
    def valid_1(passport):
        required_fields = ['byr', 'iyr', 'eyr', 'hgt', 'hcl', 'ecl', 'pid']
        if not all([key in passport for key in required_fields]):
            return False
        return True

    def valid_2(passport):
        import re
        if not valid_1(passport):
            return False

        def validate_height(x):
            match = re.match("(\d+)(cm|in)", x)
            if match is None:
                return False
            [hgt,unit] = match.groups()
            hgt = int(hgt)
            if unit == 'cm':
                return hgt >= 150 and hgt <= 193
            elif unit == 'in':
                return hgt >= 59 and hgt <= 76
            else:
                raise f"Unexpected value for hgt {hgt} unit {unit}"
        
        validations = {
            'byr': lambda x: int(x) >= 1920 and int(x) <= 2002,
            'iyr': lambda x: int(x) >= 2010 and int(x) <= 2020,
            'eyr': lambda x: int(x) >= 2020 and int(x) <= 2030,
            'hgt': validate_height,
            'hcl': lambda x: re.match("^#[0-9a-f]{6}$",x) is not None,
            'ecl': lambda x: x in 'amb blu brn gry grn hzl oth'.split(' '),
            'pid': lambda x: re.match("^[0-9]{9}$",x) is not None,
        }
        
        for field in passport:
            if field in validations:
                if not validations[field](passport[field]):
                    return False
        return True

    def day1():
        return len([p for p in passports if valid_1(p)])
    
    def day2():
        return len([p for p in passports if valid_2(p)])
    
    return day1(), day2()

print(f"Day 4: {Day4()}")

Day 4: (190, 121)


In [3]:
# https://adventofcode.com/2020/day/5
def Day5():
    def pid(d):
        row_bits = [1 if b=='B' else 0 for b in d[:7]]
        col_bits = [1 if b=='R' else 0 for b in d[7:]]
        row = bits_to_int(row_bits)
        col = bits_to_int(col_bits)
        return row*8 + col
    
    def pids():
        return [pid(d) for d in get_input(5)]
    
    def part1():
        return max(pids())
    
    def part2():
        pids_ = pids()
        for id in sorted(pids_):
            if (id+1) not in pids_:
                return id + 1
            
    return part1(), part2()
    
print(f"Day 5: {Day5()}")

Day 5: (976, 685)


In [4]:
def Day6():
    data = get_input(6)
    
    def part1():
        groups = []
        group = set()
        for line in data:
            if line == '':
                groups.append(group)
                group = set()
            else:
                for char in line:
                    group.add(char)
            
        if group is not None:
            groups.append(group)
            
        return sum([len(g) for g in groups])
            
    def part2():
        groups = []
        group = {'len': 0}
        for line in data:
            if line == '':
                groups.append(group)
                group = {'len':0}
            else:
                group['len'] += 1
                for char in line:
                    if char not in group:
                        group[char] = 1
                    else:
                        group[char] += 1
        if group is not None:
            groups.append(group)
        
        count = 0
        for group in groups:
            g_len = group['len']
            del group['len']
            count += len([k for k in group if group[k] == g_len])
        return count
            
    
    return part1(), part2()

print(f"Day 6: {Day6()}")

Day 6: (6430, 3125)


In [5]:
def Day7(data=get_input(7)):
    def parse(l):
        l = l[:-1] # drop the '.'

        import re
        container_re = r"^(.*) bags$"
        item_re = r"^(\d+) (.*) bags?$"

        container, items = l.split(' contain ')
        container = re.match(container_re, container).groups()[0]
        items_dict = {}

        if items == 'no other bags':
            pass
        else:
            items = items.split(', ')
            for i in items:
                count,name = re.match(item_re, i).groups()
                count = int(count)
                items_dict[name] = count
        return container,items_dict

    
    def part1():
        nodes = {}
        for line in data:
            container,items = parse(line)
            if container not in nodes:
                nodes[container] = set()
            for item in items:
                if not item in nodes:
                    nodes[item] = set()
                nodes[item].add(container)
        seen = set()
        check = nodes['shiny gold']
        while len(check) > 0:
            cur = check.pop()
            if cur not in seen:
                seen.add(cur)
                check |= nodes[cur]
                
        return len(seen)
            
    def part2():
        
        nodes = {}
        for line in data:
            container,items = parse(line)
            assert(container not in nodes)
            nodes[container] = items
        
        def cost(node):
            return sum([count + count*cost(child) for child,count in nodes[node].items()])
        
        return cost('shiny gold')
                    
    return part1(),part2()

ex1 = """light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags.""".split("\n")
ex2 = """shiny gold bags contain 2 dark red bags.
dark red bags contain 2 dark orange bags.
dark orange bags contain 2 dark yellow bags.
dark yellow bags contain 2 dark green bags.
dark green bags contain 2 dark blue bags.
dark blue bags contain 2 dark violet bags.
dark violet bags contain no other bags.""".split("\n")

assert(Day7(ex1) == (4,32))
assert(Day7(ex2)[1] == 126)

print(f"Day 7: {Day7()}")

Day 7: (372, 8015)


In [6]:
def Day8(data = get_input(8)):
    def parse(ins):
        op,offset = ins.split(' ')
        offset = int(offset)
        return (op,offset)
        
    
    def evaluate(instructions):
        acc,pc = (0,0)
        states = []
        seen = []
        delta = 1
        
        def process(op, offset):
            nonlocal acc
            nonlocal pc

            if op == 'acc':
                acc += offset
                pc += delta
            elif op == 'nop':
                pc += delta
            elif op == 'jmp':
                pc += delta*offset
            else:
                assert(False)
        
        while True:
            if pc in seen:
                break
            states.append( (acc, pc) )
            seen.append(pc)
            if pc >= len(instructions) or pc < 0:
                break
            ins = instructions[pc]
            process(*ins)

        return states
            
    
    def part1():
        instructions = [parse(ins) for ins in data]
        states = evaluate(instructions)
        return states[-1][0]
    
    def part2():
        instructions = [parse(ins) for ins in data]
        instruction_copies = []
        for idx,(op,offset) in enumerate(instructions):
            if op in ['jmp','nop']:
                copy = instructions.copy()
                other = 'jmp' if op == 'nop' else 'nop'
                copy[idx] = (other, offset)
                instruction_copies.append(copy)
        for copy in instruction_copies:
            states = evaluate(copy)
            (acc,pc) = states[-1]
            if pc == len(copy): # completed
                return acc

        
    
    return part1(),part2()

print(f"Day 8: {Day8()}")

Day 8: (1928, 1319)


In [7]:
def Day9(size=25):
    data = [int(i) for i in get_input(9)]

    def is_error(arr, d):
        for i in range(len(arr)):
            for j in range(i+1, len(arr)):
                if arr[i] + arr[j] == d:
                    return False
        return True
    
    def part1():
        queue = []
        for d in data:
            if len(queue) < size:
                queue.append(d)
            else:
                if is_error(queue,d):
                    return d
                queue.append(d)
                queue = queue[-size:]
        assert(False)
            
    
    def sum_from(i, data, target):
        start = i
        acc = 0
        while i < len(data):
            acc += data[i]
            if acc == target:
                return data[start:i+1]
            if acc > target:
                return None
            i += 1
        return None
    
    def part2():
        target = part1()
        for i in range(len(data)):
            found = sum_from(i, data, target)
            if found is not None:
                return min(found) + max(found)

    return part1(),part2()

print(f"Day 9: {Day9()}")

Day 9: (27911108, 4023754)


In [8]:
def Day10(data=get_input(10)):
    from collections import defaultdict
    import time
    
    data = [int(d) for d in data]
    _max = max(data)
    deltas = defaultdict(lambda: 0)
    data = sorted([0] + data + [_max + 3])

    def part1(data):
        acc = 0
        for i in range(1, len(data)):
            delta = data[i] - data[i-1]
            deltas[delta] += 1
        return deltas[1] * deltas[3]
    
    def part2(data):
        start = time.perf_counter()
        def count_tail(arr, i=0, memo=defaultdict(lambda:0)):
            if memo[i]:
                return memo[i]
            if i >= len(arr) - 1:
                return 1
            else:
                count = 0
                if i + 3 < len(arr) and arr[i+3] - arr[i] <= 3:
                    count += count_tail(arr, i + 3, memo)
                if i + 2 < len(arr) and arr[i+2] - arr[i] <= 3:
                    count += count_tail(arr, i + 2, memo)
                if i + 1 < len(arr):
                    count += count_tail(arr, i + 1, memo)
                memo[i] = count
                return count
            
        result = count_tail(data)
        print(f"Part 2 Took {time.perf_counter() - start}")
        return result
        
    
    return part1(data), part2(data)

ex1 = [16, 10, 15, 5, 1, 11, 7, 19, 6, 12, 4]
ex2 = [28, 33, 18, 42, 31, 14, 46, 20, 48, 47, 24, 23, 49, 45, 19, 38, 39, 11, 1, 32, 25, 35, 8, 17, 7, 9, 4, 2, 34, 10, 3]

print(f"Day 10: {Day10()}")

Part 2 Took 0.0001596340000000751
Day 10: (1690, 5289227976704)


In [9]:
def Day11(data = get_input(11)):
    from collections import defaultdict

    FLOOR = 0
    EMPTY = 1
    OCC   = 2
    OOB   = -1
    STR_TO_GRID = {
        'L': EMPTY,
        '.': FLOOR,
        '#': OCC        
    }
    GRID_TO_STR = {
        EMPTY: 'L',
        FLOOR: '.',
        OCC:   '#'
    }
    
    def to_grid(data):
        grid = defaultdict(lambda: OOB)
        for y,row in enumerate(data):
            for x,col in enumerate(row):
                grid[(x,y)] = STR_TO_GRID[col]
        return grid
    
    def grid_to_str(grid):
        xmax = max([point[0] for point in grid.keys()])
        ymax = max([point[1] for point in grid.keys()])
        rows = []
        for y in range(ymax):
            col = []
            for x in range(xmax):
                col.append(GRID_TO_STR[grid[(x,y)]])
            rows.append(''.join([str(c) for c in col]))
        return "\n".join(rows)
    
    def neighbors(grid, point):
        (x,y) = point
        dxs = [-1, 0, 1]
        dys = [-1, 0, 1]
        ns = []
        for dx in dxs:
            for dy in dys:
                if dx == 0 and dy == 0:
                    continue
                ns.append(grid[(x+dx,y+dy)])
        return [n for n in ns if n != OOB]
    
    def line_of_sight_neighbors(grid, point):
        (x,y) = point
        dxs = [-1, 0, 1]
        dys = [-1, 0, 1]
        ns = []
        for dx in dxs:
            for dy in dys:
                if dx == 0 and dy == 0:
                    continue
                cur_x = x
                cur_y = y
                while True:
                    cur_x += dx
                    cur_y += dy
                    seat = grid[(cur_x,cur_y)]
                    if seat == OOB:
                        break
                    if seat == FLOOR:
                        continue
                    ns.append(seat)
                    break
        return ns
    
    def tick(grid):
        new_grid = defaultdict(lambda:OOB)
        changes = 0
        for point in list(grid.keys()):
            (x,y) = point
            center = grid[point]
            if center == FLOOR:
                new_grid[point] = FLOOR
            else:
                ns = neighbors(grid, point)
                occ_count = len([n for n in ns if n == OCC])
                
                if center == EMPTY:
                    if occ_count == 0:
                        changes += 1
                        new_grid[point] = OCC
                    else:
                        new_grid[point] = EMPTY
                elif center == OCC:
                    if occ_count >= 4:
                        changes += 1
                        new_grid[point] = EMPTY
                    else:
                        new_grid[point] = OCC
                else:
                    assert(False)
        return (new_grid, changes)
    
    def line_of_sight_tick(grid):
        new_grid = defaultdict(lambda:OOB)
        changes = 0
        for point in list(grid.keys()):
            (x,y) = point
            center = grid[point]
            if center == FLOOR:
                new_grid[point] = FLOOR
            else:
                ns = line_of_sight_neighbors(grid, point)
                occ_count = len([n for n in ns if n == OCC])
                
                if center == EMPTY:
                    if occ_count == 0:
                        changes += 1
                        new_grid[point] = OCC
                    else:
                        new_grid[point] = EMPTY
                elif center == OCC:
                    if occ_count >= 5:
                        changes += 1
                        new_grid[point] = EMPTY
                    else:
                        new_grid[point] = OCC
                else:
                    assert(False)
        return (new_grid, changes)
    
    def part1(data):
        grid = to_grid(data)

        while True:
            (grid,changes) = tick(grid)
            if changes == 0:
                break
        return len([seat for seat in grid if grid[seat] == OCC])
    
    def part2(data):
        grid = to_grid(data)
        while True:
            (grid,changes) = line_of_sight_tick(grid)
            if changes == 0:
                break
        return len([seat for seat in grid if grid[seat] == OCC])
                        
    return part1(data),part2(data)

ex = """L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL""".split("\n")

print(f"Day 11: {Day11()}")

Day 11: (2476, 2257)


In [10]:
def Day12(data=get_input(12)):
    def day1(data):
        x = 0
        y = 0
        cur_dir = (1,0) # East
        
        W = (-1,0)
        E = (1,0)
        N = (0,-1)
        S = (0,1)
        dir_vectors = {
            'W': W,
            'N': N,
            'E': E,
            'S': S
        }

        def turn(cur_dir, angle):
            assert(angle % 90 == 0)
            next_dir = cur_dir
            while angle > 0: # turning right
                next_dir = turn_90(next_dir, 'R')
                angle -= 90
            while angle < 0: # turning left
                next_dir = turn_90(next_dir, 'L')
                angle += 90
            return next_dir
            
        def turn_90(cur_dir, l_or_r):
            r_dirs = [
                E, S, W, N, E
            ]
            l_dirs = [
                E, N, W, S, E
            ]
            if l_or_r == 'L':
                next_dir = l_dirs[l_dirs.index(cur_dir)+1]
            else:
                next_dir = r_dirs[r_dirs.index(cur_dir)+1]
            return next_dir

        for l in data:
            ins,amt = l[:1],int(l[1:])
            if ins == 'L' or ins == 'R':
                if ins == 'L':
                    amt *= -1
                cur_dir = turn(cur_dir, amt)
            elif ins == 'F':
                x += cur_dir[0] * amt
                y += cur_dir[1] * amt
            else:
                vector = dir_vectors[ins]
                x += vector[0] * amt
                y += vector[1] * amt

        return manhattan_distance( (0,0), (x,y) )
    
    def day2(data):
        def rotate(vec, angle):
            assert(angle % 90 == 0)
            while angle > 0:
                vec = rotate_90(vec, clockwise=True)
                angle -= 90
            while angle < 0:
                vec = rotate_90(vec, clockwise=False)
                angle += 90
            return vec
        
        def rotate_90(vec, clockwise=True):
            if clockwise:
                return (-1*vec[1],vec[0])
            else:
                return (vec[1], -1*vec[0])
            
        way = (10,-1)
        W = (-1,0)
        E = (1,0)
        N = (0,-1)
        S = (0,1)
        dir_vectors = {
            'W': W,
            'N': N,
            'E': E,
            'S': S
        }
        cur = (0,0)
        for l in data:
            ins,amt = l[:1],int(l[1:])
            if ins == 'L' or ins == 'R':
                if ins == 'L':
                    amt *= -1
                way = rotate(way, amt)
            elif ins == 'F':
                cur = (
                    cur[0] + way[0]*amt,
                    cur[1] + way[1]*amt
                )
            else: # NEWS
                vector = dir_vectors[ins]
                way = (
                    way[0] + vector[0]*amt,
                    way[1] + vector[1]*amt
                )
            
        return manhattan_distance( (0,0), cur )
        
        
    return day1(data),day2(data)

ex = """F10
N3
F7
R90
F11""".split("\n")

print(f"Day 12: {Day12()}")

Day 12: (420, 42073)


In [11]:
def Day13(data=get_input(13)):
    def day1(data):
        stamp = int(data[0])
        buses = [int(d) for d in data[1].split(',') if d != 'x']
        bus_mins = {bus:0 for bus in buses}
        for bus in buses:
            acc = 0
            while acc < stamp:
                acc += bus
            bus_mins[bus] = acc - stamp
        minwait = min(bus_mins.values())
        bus = [bus for bus in buses if bus_mins[bus] == minwait][0]
        return bus*minwait
    
    
    def day2(data):
        equivs = [(idx,int(d)) for idx,d in enumerate(data[1].split(',')) if d != 'x']

        def solve_congruence(a1,n1,a2,n2):
            """
            https://en.wikipedia.org/wiki/Chinese_remainder_theorem#Using_the_existence_construction
            """
            (bez_n1, bez_n2) = bezout_coeff(n1,n2)

            # bezout's identity:
            # bez_n1 * n1 + bez_n2 * n2 = 1

            # existence formula
            # a_2 * (bez_n1 * n1) + a_1 * (bez_n2 * n2)

            # congruence solution
            next_a = a2 * (bez_n1 * n1) + a1 * (bez_n2 * n2)
            next_n = n1*n2

            return (next_a, next_n)
        
        (a1,n1) = equivs.pop()

        while len(equivs):
            (a2,n2) = equivs.pop()
            (a1,n1) = solve_congruence(a1,n1,a2,n2)
            
        # smallest positive integer
        return n1%(a1%n1)

    return day1(data),day2(data)

ex = """100
7,13,x,x,59,x,31,19""".split("\n")

print(f"Day 13: {Day13()}")

Day 13: (102, 327300950120029)
